![Img](https://app.theheadstarter.com/static/hs-logo-opengraph.png)

# Headstarter Codebase RAG Project

![Screenshot 2024-11-25 at 7 12 58 PM](https://github.com/user-attachments/assets/48dd9de1-b4d2-4318-8f52-85ec209d8ebc)

# Install Necessary Libraries

In [ ]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

# Clone a GitHub Repo locally

In [ ]:
github_repo = 'https://github.com/Ayush-Chaudhary/BLIP'

In [ ]:
def clone_repository(repo_url):
    """Clones a GitHub repository to a temporary directory.

    Args:
        repo_url: The URL of the GitHub repository.

    Returns:
        The path to the cloned repository.
    """
    repo_name = github_repo.split('/')[-1]
    repo_path = f"/content/{repo_name}"
    Repo.clone_from(repo_url, str(repo_path))
    return repo_path


In [ ]:
path = clone_repository(github_repo)

In [ ]:
github_repo

'https://github.com/Ayush-Chaudhary/BLIP'

# Define which types of files to parse and which files / folders to ignore

In [ ]:
SUPPORTED_EXTENSIONS = {'.py', '.js', '.tsx', '.jsx', '.ipynb', '.java',
                         '.cpp', '.ts', '.go', '.rs', '.vue', '.swift', '.c', '.h', '.py'}

IGNORED_DIRS = {'node_modules', 'venv', 'env', 'dist', 'build', '.git',
                '__pycache__', '.next', '.vscode', 'vendor'}

In [ ]:
def get_file_content(file_path, repo_path):
    """
    Get content of a single file.

    Args:
        file_path (str): Path to the file

    Returns:
        Optional[Dict[str, str]]: Dictionary with file name and content
    """
    try:
      with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

      rel_path = os.path.relpath(file_path, repo_path)

      return {
          "name": rel_path,
          "content": content
      }
    except Exception as e:
      print(f"Error reading file {file_path}: {e}")
      return None


def get_main_files_content(repo_path: str):
    """
    Get content of supported code files from the local repository.

    Args:
        repo_path: Path to the local repository

    Returns:
        List of dictionaries containing file names and contents
    """
    files_content = []


    try:
      for root, _, files in os.walk(repo_path):
        if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
          continue

        for file in files:
          file_path = os.path.join(root, file)
          if os.path.splitext(file_path)[1] in SUPPORTED_EXTENSIONS:
            file_content = get_file_content(file_path, repo_path)
            if file_content:
              files_content.append(file_content)
    except Exception as e:
      print(f"Error reading files: {e}")

    return files_content

In [ ]:
files_content = get_main_files_content(path)

In [ ]:
files_content[0]

{'name': 'predict.py',
 'content': '"""\nDownload the weights in ./checkpoints beforehand for fast inference\nwget https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model*_base_caption.pth\nwget https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model*_vqa.pth\nwget https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_retrieval_coco.pth\n"""\n\nfrom pathlib import Path\n\nfrom PIL import Image\nimport torch\nfrom torchvision import transforms\nfrom torchvision.transforms.functional import InterpolationMode\nimport cog\n\nfrom models.blip import blip_decoder\nfrom models.blip_vqa import blip_vqa\nfrom models.blip_itm import blip_itm\n\n\nclass Predictor(cog.Predictor):\n    def setup(self):\n        self.device = "cuda:0"\n\n        self.models = {\n            \'image_captioning\': blip_decoder(pretrained=\'checkpoints/model*_base_caption.pth\',\n                                             image_size=384, vit=\'b

# Embeddings

In [ ]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)

In [ ]:
text = 'I like coding'
embeddings = get_huggingface_embeddings(text)

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Setting up Pinecone
**1. Create an account on [Pinecone.io](https://app.pinecone.io/)**

**2. Create a new index called "codebase-rag" and set the dimensions to 768. Leave the rest of the settings as they are.**

![Screenshot 2024-11-24 at 10 58 50 PM](https://github.com/user-attachments/assets/f5fda046-4087-432a-a8c2-86e061005238)



**3. Create an API Key for Pinecone**

![Screenshot 2024-11-24 at 10 44 37 PM](https://github.com/user-attachments/assets/e7feacc6-2bd1-472a-82e5-659f65624a88)


**4. Store your Pinecone API Key within Google Colab's secrets section, and then enable access to it (see the blue checkmark)**

![Screenshot 2024-11-24 at 10 45 25 PM](https://github.com/user-attachments/assets/eaf73083-0b5f-4d17-9e0c-eab84f91b0bc)



In [ ]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("codebase-rag")

In [ ]:
vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())

<ipython-input-15-5982ffb8f713>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())
<ipython-input-15-5982ffb8f713>:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())


In [ ]:
# Insert the codebase embeddings into Pinecone

documents = []

for file in files_content[:2]:
    doc = Document(
        page_content=f"{file['name']}\n{file['content']}",
        metadata={"source": file['name']}
    )
    documents.append(doc)

vectorstore = PineconeVectorStore.from_documents(
    documents,
    embedding=HuggingFaceEmbeddings(),
    index_name="codebase-rag",
    namespace="https://github.com/Ayush-Chaudhary/BLIP"
)


<ipython-input-27-88847094a728>:14: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding=HuggingFaceEmbeddings(),


# Perform RAG

1. Get your OpenRouter API Key [here](https://openrouter.ai/settings/keys)

2. Paste your OpenRouter Key into your Google Colab secrets, and make sure to enable permissions for it

![Image](https://github.com/user-attachments/assets/bd64c5aa-952e-4a1e-9ac0-01d8fe93aaa1)


In [ ]:
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_API_KEY")
)

In [ ]:
query = "Give an overview of the Deep learning model"

In [ ]:
raw_query_embedding = get_huggingface_embeddings(query)

In [ ]:
top_matches = pinecone_index.query(
    vector=raw_query_embedding.tolist(),
    top_k=3,
    include_metadata=True,
    namespace = 'https://github.com/Ayush-Chaudhary/BLIP'
)

In [ ]:
context = [item['metadata']['text'] for item in top_matches['matches']]

In [ ]:
context

['predict.py\n"""\nDownload the weights in ./checkpoints beforehand for fast inference\nwget https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model*_base_caption.pth\nwget https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model*_vqa.pth\nwget https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_retrieval_coco.pth\n"""\n\nfrom pathlib import Path\n\nfrom PIL import Image\nimport torch\nfrom torchvision import transforms\nfrom torchvision.transforms.functional import InterpolationMode\nimport cog\n\nfrom models.blip import blip_decoder\nfrom models.blip_vqa import blip_vqa\nfrom models.blip_itm import blip_itm\n\n\nclass Predictor(cog.Predictor):\n    def setup(self):\n        self.device = "cuda:0"\n\n        self.models = {\n            \'image_captioning\': blip_decoder(pretrained=\'checkpoints/model*_base_caption.pth\',\n                                             image_size=384, vit=\'base\'),\n            \

In [ ]:
augmented_query = f"""
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know,
don't try to make up an answer.

{context}

Question: {query}
Helpful Answer:
"""

In [ ]:
system_query = ''' Answer the question i have based on the content provided. Consider all the given content'''

llm_response = client.chat.completions.create(
    model = 'deepseek/deepseek-r1:free',
    messages=[
        {'role': 'system', 'content': system_query},
        {'role': 'user', 'content': augmented_query}
    ]
)

response = llm_response.choices[0].message.content

In [ ]:
print(response)

The provided code implements the **BLIP (Bootstrapping Language-Image Pre-training)** model, designed for vision-language tasks. Here's a concise overview:

### Key Components:
1. **Tasks Supported**:
   - **Image Captioning**: Generates textual descriptions for input images.
   - **Visual Question Answering (VQA)**: Answers natural language questions about images.
   - **Image-Text Matching**: Evaluates alignment between an image and a text caption.

2. **Model Architecture**:
   - **Vision Transformer (ViT)**: Encodes images into visual features.
   - **Text Transformer**: Processes text inputs (captions/questions).
   - **Task-Specific Heads**:
     - **Decoder**: Generates captions (for image captioning).
     - **VQA Head**: Produces answers via text generation.
     - **ITM (Image-Text Matching) Head**: Predicts alignment probability between image-text pairs.
     - **ITC (Image-Text Contrastive) Head**: Measures cosine similarity between image and text embeddings.

3. **Training